<a href="https://colab.research.google.com/github/Mi-cha-ela/Retail_demand_analysis/blob/main/Feature_Engineering_Woche_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries importieren + File Upload

# 📌 Woche 2 – User-Level Feature Engineering

In dieser Woche kombiniere ich Sessions und Trips, bereinige Daten
und baue einen User-Level-Datensatz.

Wichtig:
- Sessions = jede Interaktion
- Trips = tatsächliche Buchungen
- Ein User kann viele Sessions und viele Trips haben
- Am Ende brauche ich alles aggregiert pro User (eine Zeile = ein User)

Ziele:
✔ Outlier bereinigen  
✔ Zeiten korrekt berechnen  
✔ Trips pro User bestimmen  
✔ Start eines User-Level-Datasets für die Segmentierung


In [2]:
import pandas as pd
import numpy as np
from google.colab import files

print("📁 Bitte lade zuerst session_base_clean.csv hoch")
uploaded = files.upload()

session_filename = list(uploaded.keys())[0]
df_sessions = pd.read_csv(session_filename)
print("session_base_clean.csv geladen ✔️")

print("📁 Bitte lade jetzt not_canceled_trips.csv hoch")
uploaded2 = files.upload()

trips_filename = list(uploaded2.keys())[0]
df_trips = pd.read_csv(trips_filename)
print("not_canceled_trips.csv geladen ✔️")


📁 Bitte lade zuerst session_base_clean.csv hoch


Saving not_canceled_trips.csv to not_canceled_trips.csv
session_base_clean.csv geladen ✔️
📁 Bitte lade jetzt not_canceled_trips.csv hoch


Saving session_base_clean.csv to session_base_clean.csv
not_canceled_trips.csv geladen ✔️


In [3]:
df_sessions.head(), df_trips.head()

(                                session_id  user_id  \
 0  570279-7d86b66f8f0b468c90d9553291a4a281   570279   
 1  541084-912b0968a57647b9955d46f660febd07   541084   
 2  512201-c703668166734621b079fc8a366db203   512201   
 3  517491-49ed255542d142c4b2b6b54c20201e36   517491   
 4  488865-be40c5dcccd44022bd701f93ed83c929   488865   
 
                                    trip_id        session_start  \
 0  570279-c48e05a90f9c44e8beb667ba9cc1f8a0  2023-05-19 19:32:00   
 1  541084-2855d56d85854717a7eacf9a63dce35f  2023-07-11 05:27:00   
 2  512201-d198cb30dbe945a58808812558288792  2023-01-05 15:20:00   
 3  517491-f17c9a923a054db2955c3f56a7d8486e  2023-01-08 15:23:00   
 4  488865-1154619ac9b34950b501ead668e4e825  2023-01-25 15:56:00   
 
            session_end  session_duration  page_clicks  flight_discount  \
 0  2023-05-19 19:34:50             170.0           23            False   
 1  2023-07-11 05:28:30              90.0           12             True   
 2  2023-01-05 15:22:34    

In [4]:
# Datumsspalten automatisch in Timestamp konvertieren
for df in [df_sessions, df_trips]:
    for col in df.columns:
      if"time" in col or "date" in col:
        df[col] = pd.to_datetime(df[col], errors="coerce")



In [5]:
# Empty Sessions markieren
df_sessions["is_empty_session"] = df_sessions["trip_id"].isna().astype(int)



In [6]:
# Stornierte Trips rekonstruieren
# Session_base enthält alle trips,not_canveled enthält nur nicht stonierte
not_canceled_ids = df_trips["trip_id"].dropna().unique()
all_trip_ids = df_sessions["trip_id"].dropna().unique()

canceled_ids = np.setdiff1d(all_trip_ids, not_canceled_ids)
print("Gefundene Stornierungen:", len(canceled_ids))


Gefundene Stornierungen: 4320


In [7]:
# Session markieren die zu einem canceled Trip gehören
df_sessions["is_canceled_trip"] = df_sessions["trip_id"].isin(canceled_ids).astype(int)


In [8]:
# User_id in Trios zurückmergen!(Kritische Stelle)
df_trips = df_trips.merge(
    df_sessions[["session_id", "user_id"]],
    on="session_id",
    how="left"
)


In [9]:
# Datumswerte in echtes Datumformat umgewandelt
# um Zeitdifferenzen berechnen zu können,wurden diese Spalten konvertiert
df_trips["check_in_time"] = pd.to_datetime(df_trips["check_in_time"], errors="coerce")
df_trips["departure_time"] = pd.to_datetime(df_trips["departure_time"], errors="coerce")
df_trips["session_end"] = pd.to_datetime(df_trips["session_end"], errors="coerce")

df_trips[["check_in_time", "departure_time", "session_end"]].dtypes




,0
check_in_time,datetime64[ns]
departure_time,datetime64[ns]
session_end,datetime64[ns]


In [10]:
def calc_time_after_booking(row):

    session_end = row["session_end"]

    # Wenn session_end fehlt → kein Wert möglich
    if pd.isna(session_end):
        return None

    # Prüfe zuerst auf Flug
    if row.get("flight_booked") == True:
        dep = row.get("departure_time")
        if isinstance(dep, pd.Timestamp):
            return (dep - session_end).days
        else:
            return None

    # Prüfe dann auf Hotel
    checkin = row.get("check_in_time")
    if isinstance(checkin, pd.Timestamp):
        return (checkin - session_end).days

    return None


In [11]:
df_trips["time_after_booking_days"] = df_trips.apply(calc_time_after_booking, axis=1)
df_trips["time_after_booking_days"].head()


,time_after_booking_days
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [12]:
# User Tabelle erzeugen
user_df = pd.DataFrame()
user_df["user_id"] = df_sessions["user_id"].unique()
user_df = user_df.sort_values("user_id").reset_index(drop=True)


# Total number of session
session_counts = df_sessions.groupby("user_id")["session_id"].count()
user_df["num_sessions"] = user_df["user_id"].map(session_counts)


In [13]:
# Empty sessions erkennen (Sessions ohne Trip und ohne Buchung)
empty_flag = (df_sessions["trip_id"].isna()) & \
             (df_sessions["flight_booked"] == False) & \
             (df_sessions["hotel_booked"] == False)

empty_session_counts = df_sessions[empty_flag].groupby("user_id")["session_id"].count()

user_df["num_empty_sessions"] = user_df["user_id"].map(empty_session_counts)
user_df["num_empty_sessions"] = user_df["num_empty_sessions"].fillna(0)


In [15]:
# Falls beim Merge user_id_x und user_id_y entstanden sind:
if "user_id_x" in df_trips.columns:
    df_trips["user_id"] = df_trips["user_id_x"]
    df_trips = df_trips.drop(columns=["user_id_x", "user_id_y"], errors="ignore")

if "user_id_x" in df_sessions.columns:
    df_sessions["user_id"] = df_sessions["user_id_x"]
    df_sessions = df_sessions.drop(columns=["user_id_x", "user_id_y"], errors="ignore")

print(df_trips.columns)
print(df_sessions.columns)


Index(['session_id', 'trip_id', 'session_start', 'session_end', 'page_clicks',
       'cancellation', 'flight_booked', 'hotel_booked', 'flight_discount',
       'hotel_discount', 'flight_discount_amount', 'hotel_discount_amount',
       'origin_airport', 'destination_airport', 'seats', 'checked_bags',
       'trip_airline', 'base_fare_usd', 'departure_time', 'return_time',
       'hotel_name', 'nights', 'rooms', 'check_in_time', 'check_out_time',
       'hotel_per_room_usd', 'birthdate', 'gender', 'married', 'home_country',
       'home_city', 'home_airport', 'sign_up_date', 'session_duration',
       'time_after_booking_days', 'user_id'],
      dtype='object')
Index(['session_id', 'user_id', 'trip_id', 'session_start', 'session_end',
       'session_duration', 'page_clicks', 'flight_discount',
       'flight_discount_amount', 'hotel_discount', 'hotel_discount_amount',
       'flight_booked', 'hotel_booked', 'cancellation', 'birthdate', 'gender',
       'married', 'has_children', 'home

In [16]:
# Number of completed trips(Hotels/Flights)
trip_counts = df_trips.groupby("user_id")["trip_id"].nunique()
user_df["num_trips"] = user_df["user_id"].map(trip_counts).fillna(0)



In [17]:
# Ausgaben berechnen
df_trips["money_spent_flight"] = df_trips["base_fare_usd"].fillna(0)
df_trips["money_spent_hotel"] = df_trips["hotel_per_room_usd"].fillna(0) * df_trips["nights"].fillna(0)

total_flight_money = df_trips.groupby("user_id")["money_spent_flight"].sum()
total_hotel_money = df_trips.groupby("user_id")["money_spent_hotel"].sum()

user_df["total_spent_flight"] = user_df["user_id"].map(total_flight_money).fillna(0)
user_df["total_spent_hotel"] = user_df["user_id"].map(total_hotel_money).fillna(0)


In [18]:
# Time-after-booking aggregiern
mean_booking_time = df_trips.groupby("user_id")["time_after_booking_days"].mean()
user_df["avg_time_after_booking"] = user_df["user_id"].map(mean_booking_time)


In [19]:
# Datei speichern
user_df.to_csv("user_level_dataset.csv", index=False)
print("user_level_dataset.csv gespeichert!")


user_level_dataset.csv gespeichert!
